# Hybrid Demucs

Feel free to use the Colab version:
https://colab.research.google.com/drive/1dC9nVxk3V_VPjUADsnFu8EiT-xnU1tGH?usp=sharing

In [ ]:
!pip install -U demucs
# or for local development, if you have a clone of Demucs
# pip install -e .

     ---------------------------------------- 1.2/1.2 MB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for demucs: filename=demucs-4.0.1-py3-none-any.whl size=78398 sha256=17f0c32c991c587348c337d9706b44ef89bffd5db6d17f009a8150f574a88f57
  Stored in directory: c:\users\usuario\appdata\local\pip\cache\wheels\bb\1d\17\2263f47df7a597fde6a99a95621556c8ede8aaecc38841b2d9
Successfully built demucs
  Attempting uninstall: demucs
    Found existing installation: demucs 4.0.0
    Uninstalling demucs-4.0.0:
      Successfully uninstalled demucs-4.0.0


In [ ]:
# You can use the `demucs` command line to separate tracks
!demucs test.mp3

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in C:\Users\Usuario\AppData\Local\Programs\Microsoft VS Code\separated\htdemucs


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to C:\Users\Usuario/.cache\torch\hub\checkpoints\955717e8-8726e21a.th

  0%|          | 0.00/80.2M [00:00<?, ?B/s]
  4%|▎         | 2.98M/80.2M [00:00<00:02, 31.3MB/s]
  7%|▋         | 5.97M/80.2M [00:00<00:02, 30.1MB/s]
 11%|█         | 8.84M/80.2M [00:00<00:02, 27.6MB/s]
 14%|█▍        | 11.5M/80.2M [00:00<00:02, 26.8MB/s]
 18%|█▊        | 14.1M/80.2M [00:00<00:02, 25.1MB/s]
 21%|██        | 16.5M/80.2M [00:00<00:02, 25.0MB/s]
 24%|██▍       | 19.2M/80.2M [00:00<00:02, 26.1MB/s]
 27%|██▋       | 21.7M/80.2M [00:00<00:02, 22.5MB/s]
 30%|███       | 24.5M/80.2M [00:01<00:02, 24.2MB/s]
 34%|███▎      | 26.9M/80.2M [00:01<00:02, 24.5MB/s]
 37%|███▋      | 29.5M/80.2M [00:01<00:02, 25.3MB/s]
 40%|███▉      | 32.0M/80.2M [00:01<00:01, 25.4MB/s]
 43%|████▎     | 34.4M/80.2M [00:01<00:02, 22.4MB/s]
 46%|████▌     | 36.8M/80.2M [00:01<00:01, 23.2MB/s]
 49%|████▉     | 39.5M/80.2M [00:01<00:01, 24.5MB/

In [ ]:
# You can also load directly the pretrained models,
# for instance for the MDX 2021 winning model of Track A:
from demucs import pretrained
model = pretrained.get_model('mdx')

Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/0d19c1c6-0f06f20e.th" to C:\Users\Usuario/.cache\torch\hub\checkpoints\0d19c1c6-0f06f20e.th


  0%|          | 0.00/170M [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/7ecf8ec1-70f50cc9.th" to C:\Users\Usuario/.cache\torch\hub\checkpoints\7ecf8ec1-70f50cc9.th


  0%|          | 0.00/170M [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/c511e2ab-fe698775.th" to C:\Users\Usuario/.cache\torch\hub\checkpoints\c511e2ab-fe698775.th


  0%|          | 0.00/160M [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/demucs/mdx_final/7d865c68-3d5dd56b.th" to C:\Users\Usuario/.cache\torch\hub\checkpoints\7d865c68-3d5dd56b.th


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
# Because `model` is a bag of 4 models, you cannot directly call it on your data,
# but the `apply_model` will know what to do of it.
import torch
from demucs.apply import apply_model
x = torch.randn(1, 2, 44100 * 10)  # ten seconds of white noise for the demo
out = apply_model(model, x)[0]     # shape is [S, C, T] with S the number of sources

# So let see, where is all the white noise content is going ?
for name, source in zip(model.sources, out):
    print(name, source.std() / x.std())
# The outputs are quite weird to be fair, not what I would have expected.

In [ ]:
# now let's take a single model from the bag, and let's test it on a pure cosine
freq = 440  # in Hz
sr = model.samplerate
t = torch.arange(10 * sr).float() / sr
x = torch.cos(2 * 3.1416 * freq * t).expand(1, 2, -1)
sub_model = model.models[3]
out = sub_model(x)[0]

# Same question where does it go?
for name, source in zip(model.sources, out):
    print(name, source.std() / x.std())
    
# Well now it makes much more sense, all the energy is going
# in the `other` source.
# Feel free to try lower pitch (try 80 Hz) to see what happens !

In [ ]:
# For training or more fun, refer to the Demucs README on our repo
# https://github.com/facebookresearch/demucs/tree/main/demucs